In [62]:
from PIL import Image
import torch
import torch.nn.functional as F
import torch.nn as nn
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt
import lossclass
import torchvision
import torchvision.transforms.functional as TF
import cv2

In [41]:
modelUNet=lossclass.Models_Classification_Unet.UNet(filters=[16,32,64,128,256])
modelUNet.load_state_dict(torch.load("my_model.pt"))
modelUNet.eval()

UNet(
  (conv1_1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2_1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5_1): Con

In [42]:
modelClassification=torchvision.models.densenet121(pretrained=True)
modelClassification.features.conv0=nn.Conv2d(1,64,7,2,3)
modelClassification.classifier=nn.Linear(1024,3)
modelClassification.load_state_dict(torch.load("classification_model.pt"))
modelClassification.eval()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inp

In [43]:
classes=["meningioma","glioma","pituitary tumor"]


In [54]:
def PrepareImg(img_path):
    img=Image.open(img_path)

    trans=transforms.Compose([transforms.Grayscale(),
            transforms.Resize((512, 512))])
    img=trans(img)
    img=TF.to_tensor(img)
    img=img.numpy()
    image_tensor=torch.Tensor(img)
    image_tensor=image_tensor.view((-1,1,512,512))
    return img,image_tensor
    

In [83]:
def Create_Mask(data):
    img=data[0]
    img_tensor=data[1]
    with torch.no_grad():
        out=modelUNet(img_tensor)
        predict=modelClassification(img_tensor).argmax()

    out = out.numpy()
    out = np.resize(out, (512, 512))
    mask1=Image.fromarray(out)
    mask1=np.mat(mask1)
    img=np.mat(img)
    total_img=cv2.add(img,mask1)
    non_put_text=cv2.add(img,mask1)
    # font
    font = cv2.FONT_HERSHEY_SIMPLEX

    # org
    org = (0, 30)

    # fontScale
    fontScale = 1

    # Blue color in BGR
    color = (255, 0, 0)

    # Line thickness of 2 px
    thickness = 2

    # Using cv2.putText() method
    image = cv2.putText(total_img,classes[predict.item()] , org, font, 
                       fontScale, color, thickness, cv2.LINE_AA)

    return image

In [86]:
data=PrepareImg("data_classification/meningioma/100.png")

In [87]:
cv2.imshow("Mask",Create_Mask(data))
cv2.waitKey(0)

-1